## Overview

In this tutorial we will showcase Observability through CloudWatch using AWS Opentelemetry Instrumentation and AgentCore Observability.

Amazon Bedrock AgentCore Observability helps you trace, debug, and monitor agent performance in production environments.

AgentCore Observability provides:

* Detailed visualizations of each step in the agent workflow
* Real-time visibility into operational performance through CloudWatch dashboards
* Telemetry for key metrics such as session count, latency, duration, token usage, and error rates
* Rich metadata tagging and filtering for issue investigation
* Standardized OpenTelemetry (OTEL)-compatible format for easy integration with existing monitoring stacks


<div style="text-align:left">
    <img src="images/00-architecture.jpg" width="75%"/>
</div>

## Observability Concepts

<div style="display: flex; width: 100%;">
    <div style="flex: 50%; padding-right: 20px;">
        
#### Sessions
- **Definition**: Complete **interaction context** between user and agent
- **Scope**: Entire conversation lifecycle from initialization to termination
- **Provides**: Context persistence, state management, conversation history
- **Metrics**: Session count, duration, user engagement patterns
#### Traces
- **Definition**: Detailed record of **single request-response** cycle
- **Scope**: Complete execution path from agent invocation to response
- **Provides**: Processing steps, tool invocations, resource utilization
- **Metrics**: Request latency, processing time, error rates
#### Spans
- **Definition**: Discrete, measurable **unit of work** within execution flow
- **Scope**: Fine-grained operations with start/end timestamps
- **Provides**: Operation details, parent-child relationships, status information
- **Metrics**: Operation duration, success/failure rates, resource usage
</div>
<div style="flex: 50%; padding-left: 20px;">
    <img src="images/01-observability-concept-relationship.jpg" width="70%"/>
</div>


## Prerequisites

AgentCore Observability for Runtime works through the integration of OpenTelemetry instrumentation with your hosted agents. To enable it, ensure the following:

1. Instrument your code using the AWS Distro for Open Telemetry (ADOT) SDK (`aws-opentelemetry-distro`). Ensure it is included in your Python dependencies.
   Please note that when using the `bedrock_agentcore_starter_toolkit` to configure your agent, it takes care of the opentelemetry instrumentation for you. More details are available [here](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/observability-configure.html#observability-configure-custom).
   
2. [CloudWatch Transaction Search](https://docs.aws.amazon.com/AmazonCloudWatch/latest/monitoring/Enable-TransactionSearch.html) is enabled. This is a one-time setup to turn on Amazon CloudWatch Transaction Search.
Setup via CloudWatch Console
    1. Open the [CloudWatch Console](https://console.aws.amazon.com/cloudwatch)
    2. Navigate to **Application Signals (APM)** → **Transaction search**
    3. Choose **Enable Transaction Search**
    4. Select checkbox to **ingest spans as structured logs**
    5. (Optional) Adjust **X-Ray trace indexing** percentage (default: 1%)
    6. Choose **Save**

## lets do a test and review observability

In [115]:
from boto3.session import Session
boto_session = Session()
region = boto_session.region_name

In [110]:
def print_response(boto3_response):
    # Print the basic response details
    print("Status Code:", boto3_response['statusCode'])
    print("Content Type:", boto3_response['contentType'])
    # Handle the StreamingBody response
    if 'response' in boto3_response:
        try:
            # Read and decode the streaming response
            response_body = boto3_response['response'].read().decode('utf-8')
            print("\nResponse Body:")
            print(response_body)
        except Exception as e:
            print(f"Error reading response body: {str(e)}")

In [ ]:
#list the AgentCore runtimes and copy the agentRuntimeArn
!aws bedrock-agentcore-control list-agent-runtimes

In [ ]:
from IPython.display import Markdown, display
import json
import boto3
agentcore_client = boto3.client(
    'bedrock-agentcore',
    region_name=region
)

agent_arn = "<agentRuntimeArn>"

user_id = "700001"

session_id = f"test_observability_session_{user_id}"


prompt = f"I am customer {user_id}, what is the status of my application?"

payload=json.dumps({"prompt": prompt,
                   "user_id": user_id})

boto3_response = agentcore_client.invoke_agent_runtime(
    agentRuntimeArn=agent_arn,
    qualifier="DEFAULT",
    payload=payload,
    runtimeSessionId=session_id
)

In [111]:
print_response(boto3_response)

Status Code: 200
Content Type: application/json

Response Body:
{"result": {"role": "assistant", "content": [{"text": "Thank you for providing your customer ID. I've checked the status of your application (ID: 700001), and I can see that your application is almost complete, but there is one document still needed:\n\n**Current Application Status:**\n- ✅ Proof of income - COMPLETED\n- ❌ Employment information - MISSING\n- ✅ Proof of assets - COMPLETED\n- ✅ Credit information - COMPLETED\n\nTo move your application forward in the review process, you'll need to submit your employment information document. Once that's completed, your application can proceed to the next stage of review.\n\nWould you like information on how to submit this missing document, or do you have any other questions about your application?"}]}}


### GenAI Observability for visualization

The [CloudWatch GenAI Observability](https://docs.aws.amazon.com/AmazonCloudWatch/latest/monitoring/GenAI-observability.html) provides Bedrock AgentCore dashboard with Agent, Sessions View & Traces Views to understand the performance and execution flow of Runtime-hosted agents. You can access them by selecting GenAI Observability (Preview) in the CloudWatch console.

CloudWatch GenAI Observability provides two pre-built dashboards:
- Model Invocations – Detailed metrics on model usage, token consumption, and costs
- Amazon Bedrock AgentCore agents – Performance and decision metrics for the Amazon Bedrock agents

Click [here](https://us-west-2.console.aws.amazon.com/cloudwatch/home?region=us-west-2#/gen-ai-observability/agent-core?start=-3600000) to vavigate to the Bedrock AgentCore dashboard.You can analyze various Agents and their associated interactions under Agent view, Sessions view, and Traces view.

Click on **Sessions view** and you will be able to see the session ID that we just tested with above.


<div style="text-align:left">
    <img src="images/02-Sessions-view.jpg" width="50%"/>
</div>

Select session "test_observability_session_700001" to see a list of traces for this session. In our test there is only trace.

The trace provides end-to-end visibility into agent execution paths including LLM calls and tool usage. Select "Trajectory" to review the interconnected relationship of the spans and subsequent calls from these spans. For illustration, the screenshot below highlights execution of one of the tools available to the agent and its execution time.

<div style="text-align:left">
    <img src="images/03-trace-view.jpg" width="70%"/>
</div>


### View Logs in CloudWatch

1. Open the [CloudWatch console](https://console.aws.amazon.com/cloudwatch/)
2. In the left navigation pane, expand **Logs** and select **Log groups**
3. Search for your agent's log group:
   - Standard logs (stdout/stderr): `/aws/bedrock-agentcore/runtimes/<agent_id>-<endpoint_name>/[runtime-logs] <UUID>`
   - OTEL structured logs: `/aws/bedrock-agentcore/runtimes/<agent_id>-<endpoint_name>/runtime-logs`

### View Metrics

1. Open the [CloudWatch console](https://console.aws.amazon.com/cloudwatch/)
2. Select **Metrics** from the left navigation
3. Browse to the `bedrock-agentcore` namespace
4. Explore the available metrics

See [here](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/observability-service-provided.html) for more details on available metrics.

In [123]:
#optional

In [113]:
# Check what's stored in memory

from bedrock_agentcore.memory import MemoryClient

client = MemoryClient(region_name=region)


def list_last_k_turns(memory_id, actor_id, session_id, k):
    print("=== Memory Contents ===")
    print(f"actor_id: {actor_id}")
    print(f"session_id: {session_id}")
    
    recent_turns = client.get_last_k_turns(
        memory_id=memory_id,
        actor_id=actor_id,
        session_id=session_id,
        k=k # Adjust k to see more or fewer turns
        # branch_name="main"
    )
    
    for i, turn in enumerate(recent_turns, 1):
        print(f"Turn {i}:")
        for message in turn:
            role = message['role']
            content = message['content']['text'][:300] + "..." if len(message['content']['text']) > 300 else message['content']['text']
            print(f"  {role}: {content}")
        print()

In [121]:
memory_id = "mortgage_assistant_20250816164255-ImwbeA6D2i"

In [122]:
list_last_k_turns(memory_id, f"{user_id}", session_id, 100)

=== Memory Contents ===
actor_id: 700001
session_id: test_observability_session_700001
Turn 1:
  ASSISTANT: Thank you for providing your customer ID. I've checked the status of your application (ID: 700001), and I can see that your application is almost complete, but there is one document still needed:

**Current Application Status:**
- ✅ Proof of income - COMPLETED
- ❌ Employment information - MISSING
- ...
  ASSISTANT: I'd be happy to check the status of your loan application. To provide you with accurate information, I'll need to look up your application details using your customer ID.

Turn 2:
  USER: I am customer 700001, what is the status of my application?

